This notebook contains Avida resource plotting animation examples and associated loading utilities.  You must have ffmpeg installed in order for the animations to be render to HTML5 video.

*Only single resource experiments are allowed at this time.*

Avida must be compiled and available on the system.

For resource plots, it is recommended that an organism that is not capable of replication is used.  This will speed up the experiment run.  One recommendation would be to add nop-X to the default instruction set; change the h-divide command in the default organism to nop-X.  It is also recommended that all mutations in the avida configuration file (avida.cfg) are set to zero and that death is disabled in the configuration file as well.

In [1]:
#%matplotlib inline
# Enabling the above will show both the animation movies and the
# last plot in the animation.

import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pdb
import pandas
import warnings
import subprocess
import tempfile
import progressbar
import time

from pprint import PrettyPrinter as pprint
from matplotlib import animation, rc
from IPython.display import HTML


#rc('text',usetex=True)  # Use tex for text rendering; allows for inlined math
rc('font', family='monospace')  #Use monospace fonts
warnings.filterwarnings('ignore') # Disable warnings for clarity

Demonstration Implementation with Random Data
========================

In [2]:
def init_animation():
    """
    Initialize the animation prior to the first
    frame being drawn.  Really, this does nothing
    in this example.
    """
    ax = sb.heatmap(np.zeros(world))
    plt.title('Test')
    return ax
    

    
def animate(ndx, *fargs):
    """
    Draw a frame in the animation.
    
    :param ndx: Current top-level index into data
    :param fargs: list of parameters passed by the
                    fargs argument in the animate
                    function
    :return: the plot generated
    
    """
    plt.clf()  #Needed to clear the figure
    data = fargs[0][ndx]
    params = fargs[1]
    ax = sb.heatmap(data, cmap=cmap, **params)
    ax.tick_params(axis='both', bottom='off', labelbottom='off',
                   left='off', labelleft='off')
    plt.title(fargs[2])
    return ax
    

def build_animation(data):
    """
    Given a 3-dimension array of data, create an animation.
    
    :param data: the data to animate
    :return: the animation object
    """
    
    # Open our figure that we're going to animate upon
    fig, ax = plt.subplots()

    # To keep the colormap constant, we need to find its bounds
    # before we begin any plotting
    dmax = 0
    for dmap in data:
        m = np.amax(dmap)
        if m > dmax:
            dmax = m
    
    # fig is the figure we're operating on
    # animate is the name of the function used to draw a frame
    # frames is either an iterable or generator for the animation function
    # fargs is the arguments passed as a list of additional positional args
    #    to the animate function; here we're using it to pass the data
    #    directly as an argument as the first element, the arguments to
    #    pass as the kw dictionary to the plot function, and the third
    #    argument is the title of the chart.
    # interval is measured in milliseconds and is the time a frame shown
    # 
    # We're relying on the example in the cell below to set some globals
    # for controlling things like world size, the interval, and the color
    # map used for the heatmap.
    anim = animation.FuncAnimation(fig, animate, 
                                frames=range(0,len(data)),
                                init_func=init_animation,
                               fargs=[data, {'vmin':0, 'vmax':dmax}, 'test'], 
                               interval=interval)
    return anim


   
    
    

In [3]:
# To be used with the demonstration functions only
world = (60,60)  # Our world size
interval = 200   # ms per frame
cmap = sb.color_palette('YlGn')  # Seaborn colormap for the heatmap
data = np.random.rand(10,100,100)  # Generate our random data to animate
HTML(build_animation(data).to_html5_video())  # Build & embed animation

# Note: if the HTML function call is inside of a loop, wrap it with a
# call to display() in order for it to generate.

Avida-Specific Resource Plotting
================================

The methods and examples below will work directly with avida.

The first section shows how to load already generated data from a directory.  The second section shows how to call avida from here and grab the data generated.  Both end up animating the results.

The action PrintSpatialResources will generate data compatible with these utilities.

In [4]:
def animation_progressbar(max_value):
    '''
     The progress bar will show how much time remains along with
     a filling bar to indicate that the animation is being built.
     The pbar_widgets establishes how teh progress bar will be
     formatted.  pbar is the progress bar object itself, which
     is being passed to our animate function to update. 
     
     This should be started *outside* the build_animation function
     and finished *after* the animation is finally rendered.  Otherwise
     there'll be two bars that appear because of the timing of when
     the animation is actually generated.
    '''
    pbar_widgets = [progressbar.FormatLabel('Building Animation'),
                    '  ',
                    progressbar.Bar(), 
                    progressbar.Percentage(),
                    '  ',
                    progressbar.ETA(
                        format_zero = '%(elapsed)s elapsed',
                        format_not_started='', 
                        format_finished='',
                        format_NA='',
                        format='%(eta)s remaining')]
    prgbar = progressbar.ProgressBar(widgets=pbar_widgets, max_value=max_value)
    return prgbar


def init_frame():
    '''
    Setup the initial frame and return its axes.
    :return: the axes to plot upon
    
    For some reason this function seems to be needed lest we lose the first
    frame of our animation.  It might have something to do with the fact that
    we're using a generator to send data to the animation function.
    '''
    return plt.gca()


def gen_resource_grid(data, world_size):
    """
    This is a generator that passes the update and the cell-grid's
    resources.  The generator will run until there are no additional
    updates to plot.  *HOWEVER*, a limitation in the animation library
    requires that the number of frames in the animation be known in
    advanced.  See build_resource_animation's notes.
    
    :param data: A pandas data frame that contains the update of a
                 resource in the first column, and columns 3-end
                 contains the abundance of a resource per cell.
    :yield: A tuple of the update and the grid of resources per cell
    """
    ndx = 0
    while ndx < len(data):
        yield ndx, data.iloc[ndx,0], data.iloc[ndx, 2:].reshape(world_size).astype('float')
        ndx = ndx + 1
    

def animate_resource(info, *fargs):
    """
    Animate a single frame of a resource grid.
    
    :param info: The tuple from the generator; first value is the 
                 current frame; the second value is the update;
                 and the third value is the resouce grid to plot
    :param *fargs:  Additional positional parameters as a list as
                    specified by the FuncAnimation call by the fargs
                    argument.  In this case, fargs is a list of 
                    four values: the parameters to pass the heatmap
                    drawing function, the title of the plot, 
                    the colormap to be used in the heatmap, and
                    the progress bar to show how the animation is
                    coming along.
    :return: The plot created for the current frame.
    """
    ndx, update, grid = info  # From our generator
    params = fargs[0]    # The keyword parameters to pass the heatmap
    title = fargs[1]     # The title of the plot
    cmap = fargs[2]      # The colormap for the plot
    pbar = fargs[3]      # The progress bar
    post_fn = fargs[4]   # Post-plotting function

    plt.clf()  # Clear our figure
    ax = sb.heatmap(grid, mask=grid==0, cmap=cmap, cbar_kws={'label': 'Abundance'}, **params)
    ax.tick_params(axis='both', bottom='off', labelbottom='off',
                   left='off', labelleft='off')  # Turn off ticks
    plt.title(title)
    plt.xlabel('Update {}'.format(update))
    if post_fn is not None:
        post_fn(ax)
    pbar.update(ndx)
    return ax
    


def build_resource_animation(pdata, world_size, title, interval, cmap, pbar, post_plot_fn=None, **kw):
    """
    Animate resources from an Avida run using the PrintSpatialResources
    action.
    
    :param pdata: A Pandas dataframe containing the information from the
                  spatial resource output file.
    :param world_size: A tuple containing the (Y,X) values of the world
    :param title: The title to include with the plots in the animation
    :param interval: The animation speed in ms
    :param cmap: The seaborn colormap to be used to generate the resource
                 heatmap
    :param pbar: The progress bar to show how much of the animation is rendered
    :plot_post_fn: An optional function call to modify an axes after it is rendered
    :return: The animation object for the resource plot
    """
    fig, ax = plt.subplots()
    
    dmax = pdata.iloc[:,2:].max().max()  #Maximum abundance value
    dmin = pdata.iloc[:,2:].min().min()  #Minimum abundance value
    nframes = len(pdata)+1  #Number of frames to animate
    
    anim = animation.FuncAnimation(fig, animate_resource, init_func=init_frame, frames=gen_resource_grid(pdata, world_size),
                                   fargs=[{'vmin':dmin, 'vmax':dmax}, title, cmap, pbar, post_plot_fn], 
                                   interval=interval, save_count=nframes+1, blit=False
                                  )
    # Note: For some reason, probably because of generators, the animation itself may not
    # be rendered by this point in the code because it is not actually displayed.
    return anim
                                 

Example with Fixed Folders
------------------------
The section of code below iterates over a series of data directories inside of a single folder, animating and displaying each of them.

In [5]:
world = (60,60)  # world size in Y, X
interval = 50    # animation speed in ms
cmap = sb.cubehelix_palette(start=2, rot=0, hue=1, dark=0.10, light=0.90, gamma=1, n_colors=16)  # Heatmap colormap
data_dir_root = 'avida' # Where are all the data directories?
data_dirs = ['data']    # Which data directories should we use?
filename = 'resources.dat'  # What is the name of the 
                            # spatial resource file output in each 
                            # data directory


# Iterate over each data directory
# Load the spatial resource data from each directory
# Display the animation
# Note that the display(...) call is needed if the HTML
# function call is located inside of a loop.
for dir in data_dirs:
    path = '{}/{}/{}'.format(data_dir_root, dir, filename)
    data = pandas.read_csv(path, comment='#', skip_blank_lines=True, 
                           delimiter=' ',header=None)
    pbar = animation_progressbar(len(data))  # Build our progressbar
    pbar.start()  # The pbar has to be started here because the video won't render until after to_html5_video is called
    vid = build_resource_animation(
        data, world, 'Resources', interval, cmap, pbar).to_html5_video()
    pbar.finish()  #end our progressbar
    display(HTML(vid))  #display our video
plt.close()  #close the plot; it's now displayed
    

Building Animation  |#####################################|100%  None remaining


Functions to Plot Dynamic Experiment
----------------------------------
The code below will allow the user to setup an Avida experiment in the Python code, run the experiment, and animate the resource grid when the experiment is completed.

In [6]:
def finished_widget(code):
    '''
    Just a utility function to return a progressbar widget
    that labels whether a process exited with errors or not
    '''
    if code == 0:
        return progressbar.FormatLabel('[OK]')
    else:
        return progressbar.FormatLabel('[FAILED]')

    

    
def run_process(args, cwd, title):
    '''
    Runs the shell command args in the directory cwd with a
    spinny progress bar.
    
    :param args:  The shell command to run.
    :param cwd:   The directory to execute the command in
    :param title: Title for the spinny bar
    
    :return: None or raises an exception if the process exits
             with a non-zero exit code.  If an exception is
             thrown, print the stdout/err from the subprocess
             as well.
    '''
    
    # subprocess.PIPE can only hold about 64k worth of data before
    # it hangs the chid subprocess.  To get around this, we're writing
    # the standard output and standard error to this temporary file.
    tmp_stdout = tempfile.NamedTemporaryFile(mode='w', delete=False)
    file_stdout = open(tmp_stdout.name, 'w')
    
    # Spawn the child subprocess.  We're using Popen so we can animate
    # our spinning progressbar widget.  We're using the shell=True to
    # deal with issues trying to spawn avida properly with the command
    # line options constructed as a single string.
    # This may not work properly on Windows because reasons.  There's
    # a lot of dicussion online about how to alter it so that it does
    # work in Windows.
    proc = subprocess.Popen(args,
                   cwd=cwd, 
                   shell=True,
                   stdout=file_stdout, 
                   stderr=subprocess.STDOUT,
                   universal_newlines=True)
    
    # Set up our progressbar spinny wheel.
    pbar_widgets = [progressbar.FormatLabel(title),
                    '  ',
                    progressbar.AnimatedMarker(), 
                    '  ',
                    progressbar.FormatLabel(''),
                    '  ',
                    progressbar.ETA(
                        format_zero = '',
                        format_not_started='', 
                        format_finished='%(elapsed)s elapsed',
                        format_NA='',
                        format='')]
    pbar = progressbar.ProgressBar(widgets=pbar_widgets).start()

    # Wait for our process to finish; poll() will return the exit
    # code when it's done or None if it is still running.  The wheel
    # spins via the update().
    while proc.poll() is None:
        time.sleep(0.25)
        pbar.update()
    return_code = proc.wait()  # Grab our subprocess return code
    file_stdout.close()        # Close our subprocess's output streams file
    
    # Rest our widget to be in its final state
    pbar_widgets[2] = ' ' # Delete the spinny wheel
    pbar_widgets[4] = finished_widget(return_code)  # Describe in the pbar how we exited
    pbar.finish()  # Finish the progress bar
    
    # Handle issues if the process failed.  
    # Print the standard output / error from the process temporary
    # file out, then raise a CalledProcessError exception.
    if return_code != 0:
        with open(tmp_stdout.name) as file_stdout:
            print(file_stdout.read())
        raise subprocess.CalledProcessError(return_code, args)



def write_temp_file(contents):
    """
    Write contents to a temporary file and return the file's path.
    
    :param contents: The contents to write into the temp file
    :return: The path to the file.
    """
    f = tempfile.NamedTemporaryFile(mode='w', delete=False)
    f.write(contents)
    n = f.name
    f.close()
    return n


def run_experiment(cfg):
    """
    Run an avida experiment and return a Pandas dataframe containing
    the resource abundances per cell over the course of the experiment.
    
    The only configuration files that is generated is the environment
    file.  All other configuration settings are found in the "avida"
    folder local to the directory of this notebook.  The data directory
    and the environment file are stored on the system as temporary files.
    
    :param cfg: a dictionary containing configuration settings to set.
                Keys:
                    args   Arguments to include on the command line
                    world  A list of the X and Y, respective, world size
                    environment  Contents of the environment file
                    events       Contents of the events file
                If the file contents are not specified, the defaults
                in the avida root directory are used.
    :return: The Pandas dataframe containing the resource ata
    """
    
    cwd = cfg['cwd']    # Where is the avida working directory?
    args = cfg['args']  # Begin building our avida argument list
    
    world = cfg['world']  # Grab our world size
    
    # Add our world size.  We're requiring it because the plotting
    # function needs to know it in order to properly shape the
    # heatmap
    args = args +\
        ' -set WORLD_X {X} -set WORLD_Y {Y}'.format(X=world[0], Y=world[1])
    
    # If we need to build a new environment file, make it
    if 'environment' in cfg:
        path = write_temp_file(cfg['environment'])
        args = args + ' -set ENVIRONMENT_FILE {}'.format(path)

    # If we need to build a new events file, make it
    if 'events' in cfg:
        path = write_temp_file(cfg['events'])
        args = args + ' -set EVENT_FILE {}'.format(path)

    # Create a temporary directory to hold our avida output
    data_dir = tempfile.TemporaryDirectory()
    args = args + ' -set DATA_DIR {}'.format(data_dir.name)

    # Run avida
    run_process('./avida ' + args, cwd, 'Running Avida')
    
    # Load and return our spatial resource data
    res_path = '{}/resources.dat'.format(data_dir.name)
    data = pandas.read_csv(res_path, comment='#', skip_blank_lines=True,
                       delimiter=' ', header=None)
    return data


def plot_experiment(cfg, data_transform=None, **kw):
    """
    Run and plot the resource abundances per cell as an animation.
    
    :param cfg:  Configuration dictionary; it is also passed to
                 the run_experiment function
                 Keys used here:
                    title   Title of the plots in the animation
                    world   The X,Y coordinates of the world size

    :return: The animation object of the resources over time
    """
    world = (cfg['world'][1], cfg['world'][0])  # The Y,X size of the world; flipped to work with the plotting method
    interval = 50    # The speed of the animation in ms
    cmap = sb.cubehelix_palette(start=2, rot=0, hue=1, dark=0.10, light=0.90, gamma=1, n_colors=16)  # Heatmap colormap  
    data = run_experiment(cfg)  # Run the experiment and grab the data
    if data_transform is not None:
        data = data_transform(data)
    title = cfg['title']
    pbar = animation_progressbar(len(data))  #Initialize our progressbar
    pbar.start()  #We have to start it before build_resource_animation because of generators used to render, apparently
    anim = build_resource_animation(data, world, 
                                    title, interval, cmap, pbar, **kw).to_html5_video()
    pbar.finish() #End our progress bar
    plt.close()   # Close our plot; we don't need it anymore and if we have inlining turned on, we'll get the video and
                  # the last frame.
    return anim

def create_resource_video(cfg, **kw):
    """
    Run the experiment, animate the resource data, and return a video.
    """
    return plot_experiment(cfg, **kw)

Dynamic Experiments & Plots
-------------

We can define a dictionary that defines how to configure an Avida experiment and animate the spatial resource abudance information that is produced.  *This is currently restricted to a single resource*.

See the notes in the run_experiment function for additional details.

In [17]:
# Define a dictionary with the configuration settings
inflow_outflow = {
     'cwd':'avida',  # Where can we find Avida and the default configuration files?
     'world':[60,60], # Required for plotting purposes; overrides the configuration files
     'args':'-s 10',  # Command line arguments
     
    'environment':  
     '\
         RESOURCE food:geometry={geometry}:initial=0:inflow=1:inflowX1=5:inflowX2=15:inflowY1=5:inflowY2=15\n\
         RESOURCE food:outflowX1=30:outflowX2=40:outflowY1=30:outflowY2=30:outflow=1.0\n\
     ',  # The environment file contents; if included in the config dictionary, use the default file
         # Note that we're templating the geometry parameter so we can change it using string format(...)
         # MAKE SURE TO END EACH LINE WITH \n FOR A NEWLINE AND \ TO TELL PYTHON TO CONTINUE PARSING THE NEXT LINE
    
    'events': 
    '\
        u begin Inject default-heads.org\n\
        u 0:100:end PrintSpatialResources resources.dat\n\
        u 100000 exit\n\
    ',  # The events file; if not included in the config directory, use the default file
    
    'title':'Inflow/Outflow using grid',  # The title of the plot
}


def highlight_flows(ax):
    """
    Post-plotting function to highlight the inflow and outflow boxes.
    Inflow is cyan and outflow is magenta.
    
    :param ax: The axes plotted by the frame rendering function
    
    :return: The same axes
    """
    inflow = patches.Polygon(np.array([[5,5],[5,15],[15,15],[15,5]]), edgecolor='cyan', fill=False)
    outflow = patches.Polygon(np.array([[30,30],[30,40],[40,40],[40,30]]), edgecolor='magenta', fill=False)
    ax.add_patch(inflow)
    ax.add_patch(outflow)
    return ax
    

# Run, plot, and display animation
# We're passing through the KW parameter to the build_resource_animation function
for geom in ['torus', 'grid']:
    cfg = inflow_outflow.copy()  #Create a copy of our templated configuration
    
    # Change our title to match what we're plotting
    cfg['title'] = 'Boxed Inflow/Outflow using {geometry}\nInflow=cyan, Outflow=magenta'.format(geometry=geom)
    
    # Fill in our "templated" environment configuration
    cfg['environment'] = cfg['environment'].format(geometry=geom)  #Fill in {geometry} with an actual value
    
    # Render
    HTML(create_resource_video(cfg, **{'post_plot_fn':highlight_flows}))


Running Avida     [OK]  0:00:26 elapsed                                        
Building Animation  |#####################################|100%  None remaining
Running Avida     [OK]  0:00:24 elapsed                                        
Building Animation  |#####################################|100%  None remaining


In [9]:
"""
Gradient resource prototype test
"""
inflow_outflow = {
     'cwd':'avida',
     'args':'-s 10',  # Command line arguments
     'world':[60,60],
     
    'environment':
     r"""
     GRADIENT_RESOURCE resORN0:height=4:plateau=15:spread=25:common=1:updatestep=100000:peakx=29:peaky=29:plateau_inflow=100:initial=100
     """,  # The environment file contents
    
    'events': 
    r"""
    u begin Inject default-heads.org
    u 0:25:end PrintSpatialResources resources.dat
    u 500 exit
    """,  # The events file
    
    'title':'Inflow/Outflow using grid',  # The title of the plot
}

# Run, plot, and display animation
HTML(create_resource_video(inflow_outflow))

Running Avida     [OK]  0:00:00 elapsed                                        
Building Animation  |#####################################|100%  None remaining


Spatial Resource Error Check
===

Spatial resources are capable of having an inflow and outflow box defined within the world.  Inflow is an absolute amount, I believe, whereas outflow takes a fraction of the resource currently in the cell.  A resource configuration with an outflow of 1.0 in a bounding box should have no resource accumulate over time.  However, as the plot below shows, there is a steady increase of resource over time.  The initial low value seems to accumulate as the bulk of material from the inflow box reaches the outflow box... if I had to make a guess as to what is going on.

Such a result would happen if inflow rates are calculated after the outflow calculation is performed.

In [ ]:
inflow_outflow = {
     'cwd':'avida',
     'args':'-s 10',  # Command line arguments
     'world':[60,60],
     
    'environment':
     r"""
     RESOURCE food:geometry=grid:initial=0:inflow=1:inflowX1=5:inflowX2=15:inflowY1=5:inflowY2=15
     RESOURCE food:geometry=grid:outflowX1=30:outflowX2=40:outflowY1=30:outflowY2=30:outflow=1
     """,  # The environment file contents
    
    'events': 
    r"""
    u begin Inject default-heads.org
    u 0:100:end PrintSpatialResources resources.dat
    u 100000 exit
    """,  # The events file
    
    'title':'Inflow/Outflow using grid',  # The title of the plot
}


# ERROR CHECK
# The outflow box in the configuration above should
# remove the entire fraction of the resource within
# its boundaries.  However, it appears to accumulate
# an error over time.

plt.close()  # Close our plots if they are open.
             # We can't call subprocess he
for inflow in np.linspace(0.5,3.0,endpoint=True,num=6):
    cfg = inflow_outflow.copy()
    cfg['environment'] = '\
     RESOURCE food:geometry=grid:initial=0:inflow={inflow}:inflowX1=5:inflowX2=15:inflowY1=5:inflowY2=15\n\
     RESOURCE food:geometry=grid:outflowX1=30:outflowX2=40:outflowY1=30:outflowY2=30:outflow=1\n\
     '.format(inflow=inflow)
    data = run_experiment(cfg)
    with sb.axes_style('darkgrid'):
        with sb.color_palette("Greens",6):
            plt.plot(data.iloc[:,0], data.iloc[:,2:].min(1), label=inflow)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Inflow Rate')
plt.ylabel('Deviation from Zero')
plt.xlabel('Update')
plt.ylim(-0.3, 0.3)
plt.title('Resource Sink Error\nVarying Inflow w/ Constant Outflow=1.0')
plt.show()

for outflow in np.linspace(3.0,0.5,endpoint=True,num=6):
    cfg = inflow_outflow.copy()
    cfg['environment'] = '\
     RESOURCE food:geometry=grid:initial=0:inflow=1.0:inflowX1=5:inflowX2=15:inflowY1=5:inflowY2=15\n\
     RESOURCE food:geometry=grid:outflowX1=30:outflowX2=40:outflowY1=30:outflowY2=30:outflow={outflow}\n\
     '.format(outflow=outflow)
    data = run_experiment(cfg)
    with sb.axes_style('darkgrid'):
        with sb.color_palette("Reds_r",6):
            plt.plot(data.iloc[:,0], data.iloc[:,2:].min(1), label=outflow)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Outflow Rate')
plt.ylabel('Deviation from Zero')
plt.xlabel('Update')
plt.ylim(-0.3, 0.3)
plt.title('Resource Sink Error\nVarying Outflow w/ Constant Inflow=1.0')
plt.show()